This file is used to convert ICD-9 to ICD-10.

What is ICD?

International Classification of Diseases (ICD) codes are a set of designations used by healthcare staff to communicate diseases, symptoms, abnormal findings, and other elements of a patient's diagnosis in a way that is universally accepted by those in the medical and insurance fields. The tenth and most recent edition is known as ICD-10. 

In [4]:
import os
import pandas as pd
import numpy as np
import requests

icd9to10_df = pd.read_csv("files/icd9toicd10cmgem.csv")
icd9to10_df.head()

,icd9cm,icd10cm,flags,approximate,no_map,combination,scenario,choice_list
0,0010,A000,0,0,0,0,0,0
1,0011,A001,0,0,0,0,0,0
2,0019,A009,0,0,0,0,0,0
3,0020,A0100,10000,1,0,0,0,0
4,0021,A011,0,0,0,0,0,0


In [2]:
icd9cm_list = icd9to10_df["icd9cm"].tolist()
print(len(icd9cm_list))

icd10cm_list = icd9to10_df["icd10cm"].tolist()
print(len(icd10cm_list))

23912
23912


In [3]:
# test to print ICD-9 and ICD-10 code
num = 100
icd9_test = icd9cm_list[num]
icd10_test = icd10cm_list[num]
print(icd9_test)
print(icd10_test)

01085
A157


In [5]:
def get_icd10_code(description):
    """
    Returns the ICD-10 code(s) for the given description using the ICD API.
    """
    url = "https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search"
    params = {"sf": "code,name", "df": "code,name", "terms": description, "maxList":"20"}
    response = requests.get(url, params=params)
    data = response.json()
    return data

def get_icd9_code(description):
    """
    Returns the ICD-9 code(s) for the given description using the ICD API.
    """
    url = "https://clinicaltables.nlm.nih.gov/api/icd9cm_dx/v3/search"
    params = {"terms": description, "maxList":"20"}
    response = requests.get(url, params=params)
    data = response.json()
    return data

In [6]:
icd9_test_search = get_icd9_code(icd9_test)
print(icd9_test_search)

[1, ['01085'], None, [['010.85', ' Other primary progressive tuberculosis, tubercle bacilli not found by bacteriological examination, but tuberculosis confirmed histologically']]]


The output is a list. The first element is the number of result found. The second is the ICD-9 code list without comma. The last one is the results including the ICD-9 code and corresponding description.

In [7]:
icd_9_disease = get_icd9_code("alzheimer's")
print(icd_9_disease)

[1, ['3310'], None, [['331.0', " Alzheimer's disease"]]]


Besides, we can also search ICD-9 code by input disease description.

In [10]:
icd10_test = "R48"
icd10_test_search = get_icd10_code(icd10_test)
print(icd10_test_search)

for i in range(len(icd10_test_search[-1])):
    print(icd10_test_search[-1][i])

[6, ['R48.0', 'R48.1', 'R48.2', 'R48.3', 'R48.8', 'R48.9'], None, [['R48.0', 'Dyslexia and alexia'], ['R48.1', 'Agnosia'], ['R48.2', 'Apraxia'], ['R48.3', 'Visual agnosia'], ['R48.8', 'Other symbolic dysfunctions'], ['R48.9', 'Unspecified symbolic dysfunctions']]]
['R48.0', 'Dyslexia and alexia']
['R48.1', 'Agnosia']
['R48.2', 'Apraxia']
['R48.3', 'Visual agnosia']
['R48.8', 'Other symbolic dysfunctions']
['R48.9', 'Unspecified symbolic dysfunctions']


Use ICD-10 code to find the results. If there are several ICD-10 code match the input. All results will be listed.

In [11]:
icd10_test = "R481"
icd10_test_search = get_icd10_code(icd10_test)
print(icd10_test_search)

for i in range(len(icd10_test_search[-1])):
    print(icd10_test_search[-1][i])

[0, [], None, []]


ICD-10 input need comma. If input the ICD-10 code without proper comma position, the search result would be not expected.

In [12]:
icd10_test = "Alzheimer's"
icd10_test_search = get_icd10_code(icd10_test)
print(icd10_test_search)

for i in range(len(icd10_test_search[-1])):
    print(icd10_test_search[-1][i])

[4, ['G30.8', 'G30.9', 'G30.0', 'G30.1'], None, [['G30.8', "Other Alzheimer's disease"], ['G30.9', "Alzheimer's disease, unspecified"], ['G30.0', "Alzheimer's disease with early onset"], ['G30.1', "Alzheimer's disease with late onset"]]]
['G30.8', "Other Alzheimer's disease"]
['G30.9', "Alzheimer's disease, unspecified"]
['G30.0', "Alzheimer's disease with early onset"]
['G30.1', "Alzheimer's disease with late onset"]


ICD-10 also support search by disease description.

`icd10` package is a tool to identify ICD-10 codes and give the description of input ICD-10 code.

In [23]:
import icd10

# Convert ICD-10 code to ICD-9 code
status = icd10.exists('G30')
icd10_codes = icd10.find('G30')

print(status)
print(icd10_codes.chapter)
print(icd10_codes.block)
print(icd10_codes.block_description)
print(icd10_codes.description)
print(icd10_codes)

icd10_codes_2 = icd10.find('G30.1')
print(icd10_codes_2.chapter)
print(icd10_codes_2.block)
print(icd10_codes_2.description)
print(icd10_codes_2)

True
VI
G00-G99
Diseases of the nervous system
Alzheimer's disease
G30
VI
G00-G99
Alzheimer's disease with late onset
G30.1
